# Data Cleaning
- only keeping movies that were released (dropping anything pre-release)
- get rid of the columns that we wont use
- create binary response 

In [1]:
import pandas as pd
import numpy as np

In [52]:
# importing csv file
movies_metadata = pd.read_csv('movies_metadata.csv')

TypeError: dtype '<class 'list'>' not understood

In [21]:
# only keeping movies that were released (dropping anything pre-release)
# drops about 400 movies
movies_metadata = movies_metadata.drop(np.where(movies_metadata['status'] != 'Released')[0])

In [22]:
# dropping 9 columns that we will not use
movies_metadata = movies_metadata.drop(['adult', 'homepage', 'id', 'imdb_id', 'original_title', 'poster_path', 'status', 'video', 'overview'], axis=1)

In [23]:
# cleaning belongs_to_collection - making it binary
movies_metadata.loc[~movies_metadata.belongs_to_collection.isnull(), 'belongs_to_collection'] = 1  # not nan
movies_metadata.loc[movies_metadata.belongs_to_collection.isnull(), 'belongs_to_collection'] = 0

In [24]:
movies_metadata.columns

Index(['belongs_to_collection', 'budget', 'genres', 'original_language',
       'popularity', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'tagline',
       'title', 'vote_average', 'vote_count'],
      dtype='object')

In [25]:
print('num of movies that made money:', movies_metadata[movies_metadata['revenue']!=0].shape[0])
print('num of movies that didnt make money:', movies_metadata[movies_metadata['revenue']==0].shape[0])

num of movies that made money: 7395
num of movies that didnt make money: 37619


In [26]:
# making y binary
movies_classification = movies_metadata.copy()
movies_classification.loc[movies_metadata['revenue']!=0, 'revenue'] = 1
movies_classification.loc[movies_metadata['revenue']==0, 'revenue'] = 0

In [65]:

movies_classification.genres[0].to_list()

AttributeError: 'str' object has no attribute 'to_list'

In [58]:
movies_classification.genres = movies_classification.genres.str.replace('"', '')

In [59]:
movies_classification.genres[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [54]:
movies_classification.dtypes

belongs_to_collection     object
budget                    object
genres                    object
original_language         object
popularity                object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
tagline                   object
title                     object
vote_average             float64
vote_count               float64
dtype: object

In [53]:
# turning genres into dummy vars

# replace all the [] with ''
# make each obs a list - string.split(",")
# blank dict genre_types
# for each list in movies_classification.genres:
#     for each dict in the list:
#          if id not in genre_types:
#             genre_types |= dict

# for name in genre_types:
#     add a column to movies_classification
# make all obs vals 0

# for each name in dict:
#    if name is in obs:
#       movies_classification.loc[row, name] = 1
# do movies_classification.apply(lambda...

# for i in index:
#     for genre in movies_classification.genres[i]:
        
    
for genre in movies_classification.genres:
    print(genre)

[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]
[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]
[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]
[{'id': 35, 'name': 'Comedy'}]
[{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 18, 'name': 'Drama'}, {'id': 53, 'name': 'Thriller'}]
[{'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]
[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}]
[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 53, 'name': 'Thriller'}]
[{'id': 12, 'name': 'Adventure'}, {'id': 28, 'name': 'Action'}, {'id': 53, 'name': 'Thriller'}]
[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]
[{'id': 35, 'name

In [38]:
dummies = pd.get_dummies(movies_classification)

C:\Users\csky2\AppData\Local\Temp\ipykernel_5024\1535929241.py:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  dummies = pd.get_dummies(movies_classification)


In [39]:
dummies.shape

(45014, 155331)